<a href="https://colab.research.google.com/github/SEBADA321/ControldeCalidad/blob/master/Codon_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf
import sklearn as sk
import joblib
from google.colab import files
from sklearn.ensemble import RandomForestClassifier 
import pickle


data1 = pd.read_csv('https://drive.google.com/uc?export=download&id=1Z4v43cvTwp920NyOdboDKP7_ytC_0tBC')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
data1.head(5)

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [6]:
data1.dtypes

Kingdom         object
DNAtype          int64
SpeciesID        int64
Ncodons          int64
SpeciesName     object
                ...   
GAA            float64
GAG            float64
UAA            float64
UAG            float64
UGA            float64
Length: 69, dtype: object

In [7]:
# Verificamos que aún hay columnas que deberían ser númericas.

cols = data1.columns[data1.dtypes.eq(object)] 
cols

Index(['Kingdom', 'SpeciesName', 'UUU', 'UUC'], dtype='object')

In [8]:
data1.shape

(13028, 69)

In [9]:
data1.isna().sum().sort_values()

Kingdom    0
UCA        0
UCG        0
AGU        0
AGC        0
          ..
GUG        0
GUA        0
GUC        0
GCG        0
UGA        0
Length: 69, dtype: int64

In [10]:
# Se observa que hay datos str que no permiten manipular los demás como numéricos.
data1[['UUU', 'UUC']] = data1[['UUU', 'UUC']].apply(pd.to_numeric, errors='coerce')
null_UUU = data1['UUU'].isna().sum()
null_UUC= data1['UUC'].isna().sum()
print ("Cantidad de datos nulos en codon UUU ",null_UUU)
print ("Cantidad de datos nuls en codon UUC ",null_UUC)


Cantidad de datos nulos en codon UUU  2
Cantidad de datos nuls en codon UUC  1


In [11]:
#Lectura del dataset
 
data1.describe

data1.head(5)
 

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [12]:
def preprocess_dataset(dataset, save_metadata=True):
  
  preprocessed_dataset = dataset.copy()

  #Determinamos aquellas variables que tengan un porcentaje de registros únicos por cada variable mayor al valor de 0.7
  threshold = 0.7

  unique_percentages = dataset.nunique() / len(dataset)

  criteria = unique_percentages > threshold

  columns_to_filter = unique_percentages[criteria].keys()
  
  # Está columna contiene datos descriptivos, por tanto será transformada a str.

  preprocessed_dataset['SpeciesName'] = preprocessed_dataset['SpeciesName'].astype(str)
  preprocessed_dataset['SpeciesName']
 
  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)
 
  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  #Determinamos que variables son del tipo númerico y cuales son categóricas
  numeric_columns = list()
  categorical_columns = list()
  dictionary_of_columns_with_index_to_categorical = dict()
  dictionary_of_columns_with_categorical_to_index = dict()

  for column in preprocessed_dataset:
    #Determinamos si la variable es numérica o no
    if pd.api.types.is_numeric_dtype(preprocessed_dataset[column]):
      numeric_columns.append(column)
    else:
      #Modificamos el tipo de dato de la variable mediante "astype"
      preprocessed_dataset[column] = preprocessed_dataset[column].astype("category")

      #Verificamos si el tipo de dato de la variable fue transformado a categórico correctamente
      if not pd.api.types.is_categorical_dtype(preprocessed_dataset[column]):
        raise Exception("La columna {} no se transformó correctamente a categórica".format(column))

      dictionary_of_columns_with_index_to_categorical[column] = dict()
      dictionary_of_columns_with_categorical_to_index[column] = dict()
      
      #Indexamos los valores (categorías), sin tomar en consideración los nulos, de la variable y guardamos esa información en los diccionarios
      for index, category in enumerate(preprocessed_dataset[column].cat.categories):
        dictionary_of_columns_with_index_to_categorical[column][index] = category
        dictionary_of_columns_with_categorical_to_index[column][category] = index
      
      categorical_columns.append(column)
  
  #Reemplazamos los nulos con la mediana sólo de aquellas variables numéricas
    median_of_numeric_columns = preprocessed_dataset[numeric_columns].median()
    preprocessed_dataset[numeric_columns] = preprocessed_dataset[numeric_columns].fillna(median_of_numeric_columns)

  #Transformamos a números los valores (categorías) de las variables categóricas sin considerar los nulos
  preprocessed_dataset.replace(dictionary_of_columns_with_categorical_to_index, inplace=True)

  #Determinamos aquellas variables que tengan un porcentaje de registros nulos por cada variable mayor al valor de 0.7
  threshold = 0.7

  null_percentages = preprocessed_dataset[categorical_columns].isna().sum() / len(preprocessed_dataset)

  criteria = null_percentages > threshold

  columns_to_filter = null_percentages[criteria].keys()

  #Eliminamos las variables seleccionadas en el paso anterior
  preprocessed_dataset.drop(columns_to_filter, axis=1, inplace=True)

  #Eliminamos los registros duplicados
  preprocessed_dataset.drop_duplicates(keep="first", inplace=True)

  if save_metadata:
    #Guardamos información que consideremos imporante. **El formato pickle nos permite guardar objetos de python***
    with open("numeric_columns.pkl", "wb") as f:
      pickle.dump(numeric_columns, f)
    
    with open("categorical_columns.pkl", "wb") as f:
      pickle.dump(categorical_columns, f)
    
    with open("dictionary_of_columns_with_index_to_categorical.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_index_to_categorical, f)
    
    with open("dictionary_of_columns_with_categorical_to_index.pkl", "wb") as f:
      pickle.dump(dictionary_of_columns_with_categorical_to_index, f)
  
  return preprocessed_dataset

In [13]:
#Transformamos los datos a valores numéricos
preprocessed_dataset = preprocess_dataset(data1)

#print ("preprocessed_dataset", preprocessed_dataset.describe) 

#Se redujo la cantidad de genomas luego de eliminar duplicados a 12991, se reemplaza nulos por la mediana, moda o se puede eliminar registros.

#print (preprocessed_dataset.isna().sum().sort_values())

In [14]:
preprocessed_dataset.describe

<bound method NDFrame.describe of        Kingdom  DNAtype   Ncodons      UUU  ...      GAG      UAA      UAG      UGA
0            9        0      1995  0.01654  ...  0.04361  0.00251  0.00050  0.00000
1            9        0      1474  0.02714  ...  0.04410  0.00271  0.00068  0.00000
2            9        0      4862  0.01974  ...  0.02468  0.00391  0.00000  0.00144
3            9        0      1915  0.01775  ...  0.03446  0.00261  0.00157  0.00000
4            9        0     22831  0.02816  ...  0.03679  0.00000  0.00044  0.00131
...        ...      ...       ...      ...  ...      ...      ...      ...      ...
13023        7        0      1097  0.02552  ...  0.04102  0.00091  0.00091  0.00638
13024        7        1      2067  0.01258  ...  0.00677  0.00242  0.00097  0.01887
13025        7        1      1686  0.01423  ...  0.00297  0.00356  0.00119  0.02017
13026        7        0  40662582  0.01757  ...  0.03959  0.00099  0.00079  0.00156
13027        7        1   8998998  0.01778

In [15]:
x = preprocessed_dataset.drop("Kingdom", axis=1)
x = preprocessed_dataset.drop("DNAtype", axis=1)
y = preprocessed_dataset["Kingdom"]  
y2 = preprocessed_dataset["DNAtype"] 


In [16]:
 #Selección de Características

from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif

filtrado = SelectKBest(mutual_info_classif, k=3).fit(x, y)

In [17]:
filtrado.scores_

array([1.90047895, 0.27680145, 0.21620653, 0.24104402, 0.24737082,
       0.27206249, 0.22943154, 0.27455812, 0.48666194, 0.231203  ,
       0.21101478, 0.28174495, 0.3054379 , 0.2534046 , 0.22365916,
       0.15351643, 0.15469812, 0.24698025, 0.18385838, 0.28873599,
       0.15712813, 0.32268608, 0.17999787, 0.23301424, 0.31424586,
       0.28079256, 0.30140605, 0.25451597, 0.28251881, 0.20723033,
       0.12417998, 0.23801181, 0.20341828, 0.24161114, 0.25770843,
       0.25721252, 0.17795651, 0.19018235, 0.25135487, 0.32905842,
       0.25822079, 0.20303755, 0.12493624, 0.20469428, 0.32543597,
       0.23490359, 0.16884669, 0.34118785, 0.2322677 , 0.207553  ,
       0.20138838, 0.30560753, 0.39153006, 0.30277745, 0.2985193 ,
       0.22478505, 0.23409146, 0.44423753, 0.46537277, 0.35159569,
       0.24161868, 0.27863436, 0.3289327 , 0.19978163, 0.08013812,
       0.45225994])

In [18]:
filtrado.pvalues_

In [19]:
x_new= filtrado.transform(x)
x_new[:6]

array([[9.000e+00, 1.000e-03, 3.559e-02],
       [9.000e+00, 2.040e-03, 3.596e-02],
       [9.000e+00, 1.028e-02, 2.489e-02],
       [9.000e+00, 9.400e-03, 1.671e-02],
       [9.000e+00, 4.730e-03, 1.734e-02],
       [9.000e+00, 1.782e-02, 1.710e-03]])

In [20]:
filtrado2 = SelectKBest(mutual_info_classif, k=60).fit(x, y2)

In [21]:
filtrado2.scores_

array([0.40798052, 0.1870241 , 0.18261328, 0.15703867, 0.244928  ,
       0.23671502, 0.19294173, 0.20484347, 0.46220788, 0.17248217,
       0.2425027 , 0.19308434, 0.31637193, 0.27361223, 0.10068184,
       0.16563007, 0.21141805, 0.31894143, 0.09961517, 0.17749603,
       0.13564311, 0.28032717, 0.07295521, 0.13636744, 0.18122143,
       0.25760896, 0.3364287 , 0.1950539 , 0.20036887, 0.12171833,
       0.10026518, 0.14281111, 0.10622224, 0.20666188, 0.22847751,
       0.2174915 , 0.21540604, 0.09056685, 0.18140635, 0.24930966,
       0.2764195 , 0.15287386, 0.10924509, 0.13024666, 0.43714875,
       0.14101789, 0.15891977, 0.17582907, 0.21386591, 0.1577536 ,
       0.15826937, 0.21234438, 0.4899434 , 0.26920447, 0.2223421 ,
       0.23820783, 0.19080301, 0.30031507, 0.35853558, 0.30430198,
       0.30742723, 0.2323406 , 0.46025579, 0.16482812, 0.06313917,
       0.51043098])

In [22]:
filtrado2.pvalues_

In [23]:
#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=70)


In [24]:
print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

Train shape: (10392, 66)
Test shape: (2599, 66)


In [25]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
 'max_depth':range(5,6)
}

xgb_Model = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid = GridSearchCV (estimator= xgb_Model, param_grid = param_grid )
 
xgb_Grid.fit(x_train,y_train)
 
print ("optimal max_depth",xgb_Grid.best_estimator_.max_depth) 
print ("optimal xgb_Grid.best_score_",xgb_Grid.best_score_)
 

#clf = XGBClassifier(n_estimators=80, learning_rate=0.01,  max_depth=10,
#                    min_child_weight=2, 
#                    gamma=0,
#                    subsample=0.7,
#                    colsample_bytree=0.8,
#                    objective='binary:logistic',
#                    nthread=4,
#                    scale_pos_weight=1,
#                    seed=27,
#                    n_jobs=-1 ).fit(x_train, y_train)


optimal max_depth 5
optimal xgb_Grid.best_score_ 0.9992301677575981


In [26]:
xgb_Grid.score(x_train, y_train)

0.9998075442648191

In [27]:
xgb_Grid.score(x_test, y_test)

0.9984609465178915

In [28]:

#Mediante el método "train_test_split" usaremos el 20% de la data para probar el modelo. El parámetro "random state" nos sirve para
#poder replicar la misma separación
x_train, x_test, y2_train, y2_test = train_test_split(x, y2, test_size=0.2, random_state=70)


In [29]:
print(f"Train shape: {x_train.shape}")
print(f"Test shape: {x_test.shape}")

Train shape: (10392, 66)
Test shape: (2599, 66)


In [30]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
 'max_depth':range(9,10)
}

xgb_Model2 = XGBClassifier(learning_rate=0.01, n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
xgb_Grid2 = GridSearchCV (estimator= xgb_Model2, param_grid = param_grid )
 
xgb_Grid2.fit(x_train,y2_train)
 
print ("optimal max_depth",xgb_Grid2.best_estimator_.max_depth)
print ("optimal xgb_Grid2.best_score_",xgb_Grid2.best_score_)
 

#clf = XGBClassifier(n_estimators=80, learning_rate=0.01,  max_depth=10,
#                    min_child_weight=2, 
#                    gamma=0,
#                    subsample=0.7,
#                    colsample_bytree=0.8,
#                    objective='binary:logistic',
#                    nthread=4,
#                    scale_pos_weight=1,
#                    seed=27,
#                    n_jobs=-1 ).fit(x_train, y2_train)
xgb_Grid2.score(x_test, y2_test)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


optimal max_depth 9
optimal xgb_Grid2.best_score_ 0.9867209609269281


0.9899961523662947

In [31]:
xgb_Grid2.score(x_train, y2_train)

0.9927829099307159

In [32]:
import seaborn as sns
from sklearn.metrics import roc_auc_score
X=[]
Y=[]
train_auc= [] 
rf_Model= XGBClassifier(learning_rate=0.01,max_depth=9 ,n_estimators=80, min_child_weight=2, gamma=0, colsample_bytree=0.8, subsample= 0.7, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27, n_jobs=-1,scoring='roc_auc')
rf_Model.fit(x_train,y_train)
pred_train= rf_Model.predict(x_train)
X.append(80)
Y.append(9)
train_auc.append(roc_auc_score(y_train,pred_train)) 

ValueError: ignored

In [ ]:

y_pred_proba = rfcl.predict_proba(x_test)[::,1]
fpr, tpr, th = roc_curve(y_test,  y_pred_proba)
auc = roc_auc_score(y_test, y_pred_proba)
plt.title("Curva ROC y AUC")
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.legend(loc=4)
plt.show()

In [ ]:
# LightGBM
import lightgbm as lgb

from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import GridSearchCV


# Set params
# Scores ~0.784 (without tuning and early stopping)
params = {'boosting_type': 'gbdt',
          'max_depth' : 10,
          'objective': 'binary',
          'nthread': -1, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 2,
          'min_child_samples': 6,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# Create parameters to search
gridParams = {
    'learning_rate': [0.1],
    'n_estimators': [91],
    'num_leaves': [19],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [30], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.9],
    'reg_alpha' : [1],
    'reg_lambda' : [1.2],
    }

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params['max_depth'],
          max_bin = params['max_bin'],
          subsample_for_bin = params['subsample_for_bin'],
          subsample = params['subsample'],
          subsample_freq = params['subsample_freq'],
          min_split_gain = params['min_split_gain'],
          min_child_weight = params['min_child_weight'],
          min_child_samples = params['min_child_samples'],
          scale_pos_weight = params['scale_pos_weight'])

# To view the default model params:
mdl.get_params().keys()

# Create the grid
grid = GridSearchCV(mdl, gridParams,
                    verbose=0,
                    cv=4,
                    n_jobs=-1)
# Run the grid
grid.fit(x_train, y_train)

# Print the best parameters found
print(grid.best_params_)
print(grid.best_score_)


In [ ]:
print(grid.score(x_train, y_train))
print(grid.score(x_test, y_test))

In [ ]:
# LightGBM
import lightgbm as lgb

from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.model_selection import GridSearchCV


# Set params
# Scores ~0.784 (without tuning and early stopping)
params2 = {'boosting_type': 'gbdt',
          'max_depth' : 6,
          'objective': 'binary',
          'nthread': -1, # Updated from nthread
          'num_leaves': 64,
          'learning_rate': 0.05,
          'max_bin': 512,
          'subsample_for_bin': 200,
          'subsample': 1,
          'subsample_freq': 1,
          'colsample_bytree': 0.8,
          'reg_alpha': 5,
          'reg_lambda': 10,
          'min_split_gain': 0.5,
          'min_child_weight': 2,
          'min_child_samples': 6,
          'scale_pos_weight': 1,
          'num_class' : 1,
          'metric' : 'binary_error'}

# Create parameters to search
gridParams2 = {
    'learning_rate': [0.1,0.01],
    'n_estimators': [80,90,100],
    'num_leaves': [16,19,22],
    'boosting_type' : ['gbdt'],
    'objective' : ['binary'],
    'random_state' : [30], # Updated from 'seed'
    'colsample_bytree' : [0.65],
    'subsample' : [0.7,0.9],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2],
    'max_depth' : [6,8,10]
    }

# Create classifier to use. Note that parameters have to be input manually
# not as a dict!
mdl2 = lgb.LGBMClassifier(boosting_type= 'gbdt',
          objective = 'binary',
          n_jobs = -1, # Updated from 'nthread'
          silent = True,
          max_depth = params2['max_depth'],
          max_bin = params2['max_bin'],
          subsample_for_bin = params2['subsample_for_bin'],
          subsample = params2['subsample'],
          subsample_freq = params2['subsample_freq'],
          min_split_gain = params2['min_split_gain'],
          min_child_weight = params2['min_child_weight'],
          min_child_samples = params2['min_child_samples'],
          scale_pos_weight = params2['scale_pos_weight'])

# To view the default model params:
mdl2.get_params().keys()

# Create the grid
grid2 = GridSearchCV(mdl2, gridParams2,
                    verbose=0,
                    cv=4,
                    n_jobs=-1)
# Run the grid
grid2.fit(x_train, y2_train)

# Print the best parameters found
print(grid2.best_params_)
print(grid2.best_score_)


In [ ]:
print(grid2.score(x_train, y2_train))
print(grid2.score(x_test, y2_test))

In [33]:
preprocessed_dataset

,Kingdom,DNAtype,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,9,0,1995,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,9,0,1474,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,9,0,4862,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,9,0,1915,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,9,0,22831,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,7,0,1097,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,0.03555,0.01459,0.03920,0.01003,0.02097,0.00912,0.02005,0.01003,0.03737,0.02279,0.01367,0.01094,0.00000,0.01641,0.01094,0.01185,0.00182,0.01094,0.00456,0.01276,0.01094,0.01276,0.02097,0.01003,0.01003,0.00091,0.01732,0.01459,0.01276,0.01276,0.01641,0.00820,0.02370,0.02097,0.02917,0.03464,0.02005,0.02005,0.01459,0.02461,0.01367,0.01459,0.01914,0.03008,0.00182,0.00547,0.00547,0.00820,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638
13024,7,1,2067,0.01258,0.03193,0.01984,0.00629,0.01451,0.05322,0.07644,0.01258,0.03096,0.06386,0.03435,0.01258,0.00629,0.01258,0.01451,0.00871,0.01161,0.03338,0.02129,0.00242,0.00871,0.03338,0.01984,0.00048,0.00387,0.00774,0.01693,0.01451,0.01113,0.01016,0.02032,0.02371,0.00145,0.00339,0.01161,0.01451,0

# Descripción:

The 'Kingdom' is a 3-letter code corresponding to `xxx' in the CUTG database name: 'arc'(archaea), 'bct'(bacteria), 'phg'(bacteriophage), 'plm' (plasmid), 'pln' (plant), 'inv' (invertebrate), 'vrt' (vertebrate), 'mam' (mammal), 'rod' (rodent), 'pri' (primate), and 'vrl'(virus) sequence entries. Note that the CUTG database does not contain 'arc' and 'plm' (these have been manually curated ourselves).

The 'DNAtype' is denoted as an integer for the genomic composition in the species: 0-genomic, 1-mitochondrial, 2-chloroplast, 3-cyanelle, 4-plastid, 5-nucleomorph, 6-secondary_endosymbiont, 7-chromoplast, 8-leucoplast, 9-NA, 10-proplastid, 11-apicoplast, and 12-kinetoplast.

The species identifier ('SpeciesID') is an integer, which uniquely indicates the entries of an organism. It is an accession identifier for each different species in the original CUTG database, followed by the first item listed in each genome.

The number of codons (`Ncodons') is the algebraic sum of the numbers listed for the different codons in an entry of CUTG. Codon frequencies are normalized to the total codon count, hence the number of occurrences divided by 'Ncodons' is the codon frequencies listed in the data file.

The species' name ('SpeciesName') is represented in strings purged of `comma` (which are now replaced by `space`). This is a descriptive label of the name of the species for data interpretations.

Lastly, the codon frequencies ('codon') including 'UUU', 'UUA', 'UUG', 'CUU', etc., are recorded as floats (with decimals in 5 digits).



In [164]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
import tensorflow as tf
data1

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,100217,1995,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,100220,1474,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,100755,4862,Sweet potato leaf curl virus,0.01974,0.0218,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,100880,1915,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,100887,22831,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13023,pri,0,9601,1097,Pongo pygmaeus abelii,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,0.03555,0.01459,0.03920,0.01003,0.02097,0.00912,0.02005,0.01003,0.03737,0.02279,0.01367,0.01094,0.00000,0.01641,0.01094,0.01185,0.00182,0.01094,0.00456,0.01276,0.01094,0.01276,0.02097,0.01003,0.01003,0.00091,0.01732,0.01459,0.01276,0.01276,0.01641,0.00820,0.02370,0.02097,0.02917,0.03464,0.02005,0.02005,0.01459,0.02461,0.01367,0.01459,0.01914,0.03008,0.00182,0.00547,0.00547,0.00820,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638
13024,pri,1,9601,2067,mitochondrion Pongo pygmaeus abel

In [228]:
def ML_prepocess(data):
  ml_data = data.copy()
  ml_data.Kingdom = ml_data.Kingdom.astype('category')
  drop_col = ['SpeciesID','SpeciesName']
  print("Columnas eliminadas:", drop_col)
  ml_data.drop(drop_col,inplace=True, axis=1)

  ml_data[['UUU', 'UUC']] = ml_data[['UUU', 'UUC']].apply(pd.to_numeric, errors='coerce')
  criteria = ~ml_data.isna()
  ml_data = ml_data[criteria['UUU']]
  ml_data = ml_data[criteria['UUC']]
  return ml_data

In [233]:
ml_data = ML_prepocess(data1)
ml_data.head(5)

Columnas eliminadas: ['SpeciesID', 'SpeciesName']


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


,Kingdom,DNAtype,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0,1995,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0,1474,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0,4862,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0,1915,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0,22831,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131


In [234]:
ml_data.dtypes

Kingdom    category
DNAtype       int64
Ncodons       int64
UUU         float64
UUC         float64
             ...   
GAA         float64
GAG         float64
UAA         float64
UAG         float64
UGA         float64
Length: 67, dtype: object

In [235]:
def Create_X_Y(ml_data, target_col):
  x = ml_data.drop(target_col, axis=1)
  y = ml_data[target_col]

  #enc = OneHotEncoder(handle_unknown='ignore')
  #enc.fit(y)
  #print(enc.categories_)
  #dummy_y = enc.transform(y)

  #encoder = LabelEncoder()
  #encoder.fit(y[target_col[0]])
  #encoded_Y = encoder.transform(y[target_col[0]])
  #dummy_y = tf.keras.utils.to_categorical(encoded_Y, num_classes=y.value_counts().index.shape[0])

  lb = LabelBinarizer()
  lb.fit(y[target_col[0]])
  print(lb.classes_)
  dummy_y = lb.transform(y[target_col[0]])
  df_y_1 = pd.DataFrame(dummy_y, columns = lb.classes_)
  
  lb.fit(y[target_col[1]])
  print(lb.classes_)
  dummy_y = lb.transform(y[target_col[1]])
  df_y_2 = pd.DataFrame(dummy_y, columns = lb.classes_)
  result = pd.concat([df_y_1, df_y_2], axis=1)

  #x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=70)
  return x, df_y_1, df_y_2, result

In [240]:
target_col = ['Kingdom', 'DNAtype']
X, df_y_1, df_y_2, Y = Create_X_Y(ml_data, target_col)
Y

['arc' 'bct' 'inv' 'mam' 'phg' 'plm' 'pln' 'pri' 'rod' 'vrl' 'vrt']
[ 0  1  2  3  4  5  6  7  9 11 12]


,arc,bct,inv,mam,phg,plm,pln,pri,rod,vrl,vrt,0,1,2,3,4,5,6,7,9,11,12
0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13021,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
13022,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
13023,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
13024,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [246]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=70)
dataset = tf.data.Dataset.from_tensor_slices((x_train.values, y_train.values))
train_dataset = dataset.shuffle(len(X) + len(Y), reshuffle_each_iteration=True).batch(1)

In [251]:
def get_compiled_model():
  ml_model = tf.keras.Sequential([
    #tf.keras.layers.BatchNormalization(input_shape=(preprocessed_dataset.shape[1]-1,)),
    tf.keras.layers.Dense(20, activation='relu'),
    #tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(20, activation='relu'),
    #tf.keras.layers.Dropout(.2),
    #tf.keras.layers.Dense(10, activation='relu'),
    #tf.keras.layers.Dropout(.2),
    #tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(22, activation='sigmoid')
  ])

  ml_model.compile(optimizer='adam',
                loss='binary_crossentropy',#tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.AUC(name='auc'), tf.keras.metrics.Accuracy(name='accuracy')])
  return ml_model

In [250]:
ml_model = get_compiled_model()
hist = ml_model.fit(train_dataset, validation_data=(x_test, y_test), epochs=10)

Epoch 1/10
10420/10420 [==============================] - 16s 1ms/step - loss: 44.8346 - auc: 0.7280 - binary_accuracy: 0.8856 - val_loss: 15.7398 - val_auc: 0.7612 - val_binary_accuracy: 0.9222
Epoch 2/10
10420/10420 [==============================] - 15s 1ms/step - loss: 11.6783 - auc: 0.7894 - binary_accuracy: 0.8924 - val_loss: 29.8816 - val_auc: 0.7133 - val_binary_accuracy: 0.8996
Epoch 3/10
10420/10420 [==============================] - 15s 1ms/step - loss: 4.3725 - auc: 0.8014 - binary_accuracy: 0.9045 - val_loss: 3.3354 - val_auc: 0.7976 - val_binary_accuracy: 0.8850
Epoch 4/10
10420/10420 [==============================] - 15s 1ms/step - loss: 0.8168 - auc: 0.8664 - binary_accuracy: 0.9131 - val_loss: 0.1919 - val_auc: 0.8995 - val_binary_accuracy: 0.9278
Epoch 5/10
10420/10420 [==============================] - 15s 1ms/step - loss: 0.1909 - auc: 0.9004 - binary_accuracy: 0.9284 - val_loss: 0.1922 - val_auc: 0.8995 - val_binary_accuracy: 0.9278
Epoch 6/10
10420/10420 [=======

In [252]:
ml_model.evaluate(x_test, y_test)

82/82 [==============================] - 0s 3ms/step - loss: 0.1921 - auc: 0.8994 - binary_accuracy: 0.9278


[0.19208329916000366, 0.8994148373603821, 0.9277890920639038]